# Argumentation framework

### Unit testing

In [ ]:
from utils.iccma.index import get_graphs
from utils.test.index import test

TESTING = True

if TESTING:
    graphs = get_graphs([
        ("scc", "small"),
        ("gr", "small"),
        ("st", "small"),
    ])
    # Complete, stable and preferred
    test(
        iccma_graphs=graphs,
        solvers=["minisat"],
        semantics=["PR"],
        tasks=["SE"],
        timeout=100,  # seconds
    )
    # Grounded (must not set the solvers)
    # test(
    #     iccma_graphs=graphs,
    #     solvers=[],
    #     semantics=["GR"],
    #     tasks=["SE", "EE"],
    #     timeout=1,  # seconds
    # )
else:
    print("Skipped testing")

### Benchmarking

In [ ]:
from utils.benchmark.bar_graph import bench
from typing import List
from itertools import product
from utils.iccma.index import get_graphs, GraphType, Size
from utils.graph import Graph

TYPES: List[GraphType] = ["gr", "st", "scc"]
SIZES: List[Size] = ["small", "medium", "large"]

for type, size in product(TYPES, SIZES):
    if type == "st" and size == "large":
        continue
    bench(
        graphs=[
            Graph.from_iccma_graph(iccma_graph)
            for iccma_graph in get_graphs([(type, size)])
        ],
        solvers=[
            "minisat",
            "minisat --pr-mss",
        ],
        semantics=["PR"],
        tasks=["SE"],
        name=f"pr-{type}-{size}",
    )

In [ ]:
from utils.benchmark.line_graph import bench
from utils.problem import Problem, Task, Semantics
from typing import List
from graphs import graphs
from itertools import product


TASKS: List[Task] = ["SE", "EE", "DC", "DS"]
SEMANTICS: List[Semantics] = ["CO", "ST", "PR"]
SOLVERS = [
    "minisat",
    "manysat",
    "glucose",
    # "glucose,glucose-pre",
    "maplesat",
]

name = "windmill"
graph = graphs[name]
for task, sem in product(TASKS, SEMANTICS):
    if task == "SE" and sem == "CO":
        continue
    problem = Problem(task, sem)
    print(f"{name}: {problem}")
    bench(
        graph["x_label"],
        graph["inputs"],
        graph["gen"],
        SOLVERS,
        problem,
        name,
        save_graphs=True,
    )